In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/32 x 32 images/DCT_10.pkl', 'rb') as f:
    DCT_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/32 x 32 images/DCT_20.pkl', 'rb') as f:
    DCT_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/32 x 32 images/DCT_30.pkl', 'rb') as f:
    DCT_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/32 x 32 images/DCT_40.pkl', 'rb') as f:
    DCT_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_32x32.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
DCT_10_normalized = normalize_max_row_norm(DCT_10)
DCT_20_normalized = normalize_max_row_norm(DCT_20)
DCT_30_normalized = normalize_max_row_norm(DCT_30)
DCT_40_normalized = normalize_max_row_norm(DCT_40)

DCT_10_observations = generate_observations(img_dataset, DCT_10_normalized)
DCT_20_observations = generate_observations(img_dataset, DCT_20_normalized)
DCT_30_observations = generate_observations(img_dataset, DCT_30_normalized)
DCT_40_observations = generate_observations(img_dataset, DCT_40_normalized)

DCT_10_train, DCT_10_test = split_dataset(DCT_10_observations, train_ratio = 0.9, seed = 0)
DCT_20_train, DCT_20_test = split_dataset(DCT_20_observations, train_ratio = 0.9, seed = 0)
DCT_30_train, DCT_30_test = split_dataset(DCT_30_observations, train_ratio = 0.9, seed = 0)
DCT_40_train, DCT_40_test = split_dataset(DCT_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 5)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'DCT_32_Depth5/5 Projections'

DCT_10_proj5_weights, DCT_10_proj5_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj5', display=True)

DCT_20_proj5_weights, DCT_20_proj5_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj5', display=True)

DCT_30_proj5_weights, DCT_30_proj5_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj5', display=True)

DCT_40_proj5_weights, DCT_40_proj5_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, DCT_10_proj5_weights, DCT_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, DCT_20_proj5_weights, DCT_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, DCT_30_proj5_weights, DCT_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, DCT_40_proj5_weights, DCT_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.06313871338632372
Average test loss: 0.005042340987051527
Epoch 2/300
Average training loss: 0.024620196673605176
Average test loss: 0.0047560481576042046
Epoch 3/300
Average training loss: 0.023512551594939497
Average test loss: 0.004668490430961052
Epoch 4/300
Average training loss: 0.023034626689222123
Average test loss: 0.004602760399795241
Epoch 5/300
Average training loss: 0.022751358636551432
Average test loss: 0.004562286726716492
Epoch 6/300
Average training loss: 0.022562033312188253
Average test loss: 0.004530653908745282
Epoch 7/300
Average training loss: 0.022385004955033462
Average test loss: 0.004501940273576313
Epoch 8/300
Average training loss: 0.022244713672333292
Average test loss: 0.00452137959169017
Epoch 9/300
Average training loss: 0.022140113171603946
Average test loss: 0.004557130109518767
Epoch 10/300
Average training loss: 0.022031057058109177
Average test loss: 0.004442170473436514

In [8]:
DCT_10_proj5_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj5_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj5_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj5_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj5_psnr = average_PSNR(DCT_10_proj5_model, DCT_10_proj5_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj5_psnr = average_PSNR(DCT_20_proj5_model, DCT_20_proj5_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj5_psnr = average_PSNR(DCT_30_proj5_model, DCT_30_proj5_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj5_psnr = average_PSNR(DCT_40_proj5_model, DCT_40_proj5_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.83
Average PSNR for Projection Layer 1 across 2500 images: 28.15
Average PSNR for Projection Layer 2 across 2500 images: 28.37
Average PSNR for Projection Layer 3 across 2500 images: 28.50
Average PSNR for Projection Layer 4 across 2500 images: 28.61
Average PSNR for Projection Layer 0 across 2500 images: 28.55
Average PSNR for Projection Layer 1 across 2500 images: 29.06
Average PSNR for Projection Layer 2 across 2500 images: 29.63
Average PSNR for Projection Layer 3 across 2500 images: 29.95
Average PSNR for Projection Layer 4 across 2500 images: 30.16
Average PSNR for Projection Layer 0 across 2500 images: 28.98
Average PSNR for Projection Layer 1 across 2500 images: 30.03
Average PSNR for Projection Layer 2 across 2500 images: 30.61
Average PSNR for Projection Layer 3 across 2500 images: 30.79
Average PSNR for Projection Layer 4 across 2500 images: 31.08
Average PSNR for Projection Layer 0 across 2500 images: 29.85
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'DCT_32_Depth5/15 Projections'

DCT_10_proj15_weights, DCT_10_proj15_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj15', display=True)

DCT_20_proj15_weights, DCT_20_proj15_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj15', display=True)

DCT_30_proj15_weights, DCT_30_proj15_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj15', display=True)

DCT_40_proj15_weights, DCT_40_proj15_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, DCT_10_proj15_weights, DCT_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, DCT_20_proj15_weights, DCT_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, DCT_30_proj15_weights, DCT_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, DCT_40_proj15_weights, DCT_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 1.1330272437466515
Average test loss: 0.006804341605140103
Epoch 2/300
Average training loss: 0.282161224020852
Average test loss: 0.005253058234436644
Epoch 3/300
Average training loss: 0.17864352610376147
Average test loss: 0.00479353008295099
Epoch 4/300
Average training loss: 0.136078941391574
Average test loss: 0.0047023117937561535
Epoch 5/300
Average training loss: 0.11347179792324702
Average test loss: 0.004673058655526903
Epoch 6/300
Average training loss: 0.10054848439163631
Average test loss: 0.004607108553250631
Epoch 7/300
Average training loss: 0.0915685053202841
Average test loss: 0.004573599487129185
Epoch 8/300
Average training loss: 0.08530502916706934
Average test loss: 0.004589980493817064
Epoch 9/300
Average training loss: 0.08076136020157072
Average test loss: 0.004503343797599276
Epoch 10/300
Average training loss: 0.07731829083628125
Average test loss: 0.004472379236585564
Epoch 11/300
A

In [10]:
DCT_10_proj15_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj15_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj15_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj15_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj15_psnr = average_PSNR(DCT_10_proj15_model, DCT_10_proj15_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj15_psnr = average_PSNR(DCT_20_proj15_model, DCT_20_proj15_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj15_psnr = average_PSNR(DCT_30_proj15_model, DCT_30_proj15_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj15_psnr = average_PSNR(DCT_40_proj15_model, DCT_40_proj15_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.61
Average PSNR for Projection Layer 1 across 2500 images: 27.75
Average PSNR for Projection Layer 2 across 2500 images: 27.93
Average PSNR for Projection Layer 3 across 2500 images: 28.03
Average PSNR for Projection Layer 4 across 2500 images: 28.09
Average PSNR for Projection Layer 5 across 2500 images: 28.25
Average PSNR for Projection Layer 6 across 2500 images: 28.15
Average PSNR for Projection Layer 7 across 2500 images: 28.28
Average PSNR for Projection Layer 8 across 2500 images: 28.29
Average PSNR for Projection Layer 9 across 2500 images: 28.34
Average PSNR for Projection Layer 10 across 2500 images: 28.44
Average PSNR for Projection Layer 11 across 2500 images: 28.57
Average PSNR for Projection Layer 12 across 2500 images: 28.65
Average PSNR for Projection Layer 13 across 2500 images: 28.72
Average PSNR for Projection Layer 14 across 2500 images: 28.74
Average PSNR for Projection Layer 0 across 2500 images: 28.09
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'DCT_32_Depth5/30 Projections'

DCT_10_proj30_weights, DCT_10_proj30_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj30', display=True)

DCT_20_proj30_weights, DCT_20_proj30_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj30', display=True)

DCT_30_proj30_weights, DCT_30_proj30_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj30', display=True)

DCT_40_proj30_weights, DCT_40_proj30_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, DCT_10_proj30_weights, DCT_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, DCT_20_proj30_weights, DCT_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, DCT_30_proj30_weights, DCT_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, DCT_40_proj30_weights, DCT_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 11.611238171047635
Average test loss: 0.0058268382638278935
Epoch 2/300
Average training loss: 4.36401690059238
Average test loss: 0.005199668066783084
Epoch 3/300
Average training loss: 3.242692593044705
Average test loss: 0.0049144512642588884
Epoch 4/300
Average training loss: 2.455190651787652
Average test loss: 0.005047742110987504
Epoch 5/300
Average training loss: 1.8665049104690552
Average test loss: 0.007264497928735283
Epoch 6/300
Average training loss: 1.4589252036412557
Average test loss: 0.004670723306636016
Epoch 7/300
Average training loss: 1.2261941148969862
Average test loss: 0.004662390823372537
Epoch 8/300
Average training loss: 1.021850704352061
Average test loss: 0.00458984771081143
Epoch 9/300
Average training loss: 0.8324774016274347
Average test loss: 0.004619019539405902
Epoch 10/300
Average training loss: 0.6846325942145454
Average test loss: 0.004617574618508418
Epoch 11/300
Average t

In [12]:
DCT_10_proj30_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj30_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj30_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj30_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj30_psnr = average_PSNR(DCT_10_proj30_model, DCT_10_proj30_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj30_psnr = average_PSNR(DCT_20_proj30_model, DCT_20_proj30_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj30_psnr = average_PSNR(DCT_30_proj30_model, DCT_30_proj30_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj30_psnr = average_PSNR(DCT_40_proj30_model, DCT_40_proj30_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.63
Average PSNR for Projection Layer 1 across 2500 images: 27.41
Average PSNR for Projection Layer 2 across 2500 images: 27.93
Average PSNR for Projection Layer 3 across 2500 images: 27.81
Average PSNR for Projection Layer 4 across 2500 images: 27.96
Average PSNR for Projection Layer 5 across 2500 images: 28.22
Average PSNR for Projection Layer 6 across 2500 images: 28.22
Average PSNR for Projection Layer 7 across 2500 images: 28.15
Average PSNR for Projection Layer 8 across 2500 images: 28.37
Average PSNR for Projection Layer 9 across 2500 images: 28.34
Average PSNR for Projection Layer 10 across 2500 images: 28.28
Average PSNR for Projection Layer 11 across 2500 images: 28.40
Average PSNR for Projection Layer 12 across 2500 images: 28.43
Average PSNR for Projection Layer 13 across 2500 images: 28.45
Average PSNR for Projection Layer 14 across 2500 images: 28.54
Average PSNR for Projection Layer 15 across 2500 images: 28.52
Av